In [1]:
pip install txtinstruct > /dev/null

txtinstruct consists of three components to help train instruction-following models. 

## 3-components
 
**Statement generation** models create a statement from a context. This statement can be a question or request to describe a concept depending on the model.

**Knowledge source** for pulling context. An example knowledge source used in this notebook is a txtai embeddings index of the full Wikipedia dataset.

**Large language model (LLM)** for translating source statements into target statements. A prompt is used in combination with the knowledge source context to generate the target text

In [1]:
from datasets import load_dataset

from txtinstruct.models import StatementGenerator

# Load SQuAD dataset
dataset = load_dataset("squad", split="train")

# Train model
generator = StatementGenerator()

In [2]:
model, tokenizer = generator(
    "google/flan-t5-small",
    dataset,
    "sequence-sequence",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=128 // 16,
    num_train_epochs=0.001,
    logging_steps=100,
)
#Note that we only trained the model for a fraction of an epoch 
#for expediency. Under normal circumstances, num_train_epochs 
#would be at least 3.

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [3]:
from txtai.pipeline import Sequences

# Load statement generation model
statements = Sequences((model, tokenizer))

# Run example prompt
statements("""Generate a question using the context below.
### Context:
Hugging face is an open-source platform for hosting 
all kind of AI language models.""")
     

'What is an open-source platform for hosting all kinds of AI language models?'

In [ ]:
from txtai.embeddings import Embeddings
from txtinstruct.data import DatasetBuilder

# Load embeddings
embeddings = Embeddings()
embeddings.load(provider="huggingface-hub", 
                container="neuml/txtai-wikipedia")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# Query templates
templates = [
    "Tell me about {text}",
    "Give an explanation on {text}",
    "Provide a quick summary on {text}",
    "Explain {text} in simple terms",
    "Describe {text}"
]

In [ ]:
# Build dataset
builder = DatasetBuilder(Sequences("google/flan-t5-base"), 
                         statements, 
                         templates)
builder(
    embeddings.search("SELECT id, text FROM txtai WHERE similar('machine learning') AND percentile >= 0.99 LIMIT 5"),
    5,
    "data.json"
)

In [ ]:
import json

from txtinstruct.models import Instructor

# Read in generated dataset
with open("data.json", encoding="utf-8") as f:
    data = json.load(f)

# Instruction-tune model
instructor = Instructor()
model, tokenizer = instructor(
    "google/flan-t5-small", 
    data,
    "sequence-sequence",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=128 // 8,
    num_train_epochs=0.001,
    logging_steps=100,
)